In [1]:
import pandas as pd
import numpy as np

# Load data from Excel files
client_list = pd.read_excel("Client_List.xlsx")
compliance_user_report = pd.read_excel("ComplianceUserReport.xlsx")
user_aml_rating_report = pd.read_excel("UserAmlRatingReport.xlsx")
fund_deposits = pd.read_excel("FundDeposits.xlsx")
fund_withdrawals = pd.read_excel("FundWithdrawals.xlsx")
client_info_report = pd.read_excel("clientInfoReport.xlsx")

# Assume we have the latest FX rates in a dictionary
# For example, let's use some mock FX rates
fx_rates = {
    'USD': 1.31,  # 1 USD = 1.35 SGD
    'SGD': 1.0, # 1 SGD = 1 SGD
    'HKD': 0.17,
    'CAD': 0.94,
    'EUR': 1.49,
    'AUD': 0.84,
    'GBP': 1.76
}

# Merge dataframes based on client_id
master_df = client_list.merge(user_aml_rating_report, left_on='client_id', right_on='Kristal Client ID', how='left') \
                       .merge(client_info_report, on='client_id', how='left')

# Select required columns
master_df = master_df[['client_id', 'country_of_onboarding', 'country_of_residence_x', 'billing_type_x', 'kyc_status_x', 'Assigned to entity', 'Rating', 'Reason', 'quantum_of_wealth']]

# Rename columns for consistency
master_df.columns = ['client_id', 'country_of_onboarding', 'country_of_residence', 'billing_type', 'kyc_status', 'assigned_to_entity', 'rating', 'reason', 'quantum_of_wealth']

# Create derived column 'qow' based on 'quantum_of_wealth'
def calculate_qow(quantum_of_wealth):
    if quantum_of_wealth == 'LESS_THAN_0_5M':
        return 250000
    elif quantum_of_wealth == 'BETWEEN_1M_TO_2M':
        return 750000
    elif quantum_of_wealth == 'BETWEEN_2M_TO_5M':
        return 2500000
    elif quantum_of_wealth == 'BETWEEN_0_5M_TO_1M':
        return 750000
    elif quantum_of_wealth == 'MORE_THAN_5M':
        return 7500000
    elif quantum_of_wealth == 'LESS_THAN_10LAKHS':
        return 8000
    elif quantum_of_wealth == 'MORE_THAN_1CR':
        return 100000
    elif quantum_of_wealth == 'BETWEEN_50LAKHS_TO_1CR':
        return 120000
    elif quantum_of_wealth == 'BETWEEN_25LAKHS_TO_50LAKHS':
        return 60000
    elif quantum_of_wealth == 'BETWEEN_10LAKHS_TO_25LAKHS':
        return 20000
    else:
        return np.nan

master_df['qow'] = master_df['quantum_of_wealth'].apply(calculate_qow)

# Filter completed deposits and withdrawals
completed_deposits = fund_deposits[fund_deposits['status_text'] == 'COMPLETED']
completed_withdrawals = fund_withdrawals[fund_withdrawals['internal_status'] == 'COMPLETED']

# Strip time component and convert to datetime
completed_deposits['requested_time'] = pd.to_datetime(completed_deposits['requested_time'].str[:10], errors='coerce')

completed_deposits['requested_time'] = pd.to_datetime(completed_deposits['requested_time'], format='%d/%m/%Y', errors='coerce')

completed_withdrawals['request_time'] = pd.to_datetime(completed_withdrawals['request_time'].str[:10], errors='coerce')

completed_withdrawals['request_time'] = pd.to_datetime(completed_withdrawals['request_time'], format='%d/%m/%Y', errors='coerce')

# Identify the latest month in the data
#latest_month = max(completed_deposits['requested_time'].max(), completed_withdrawals['request_time'].max()).to_period('M')
deposits_max = completed_deposits['requested_time'].max()
withdrawals_max = completed_withdrawals['request_time'].max()

latest_month = max(deposits_max, withdrawals_max).to_period('M')


# Filter data for the last 12 months
start_month = latest_month - 11
completed_deposits = completed_deposits[completed_deposits['requested_time'].dt.to_period('M').between(start_month, latest_month)]
completed_withdrawals = completed_withdrawals[completed_withdrawals['request_time'].dt.to_period('M').between(start_month, latest_month)]

# Convert requested_amount to SGD in deposits
completed_deposits['requested_amount_SGD'] = completed_deposits.apply(
    lambda row: row['requested_amount'] * fx_rates.get(row['currency'], 1), axis=1
)

# Convert request_amount_USD to SGD in withdrawals
#completed_withdrawals['request_amount_SGD'] = completed_withdrawals['request_amount_USD'] * fx_rates['USD']
completed_withdrawals['requested_amount_SGD'] = completed_withdrawals.apply(
    lambda row: row['request_amount'] * fx_rates.get(row['request_currency'], 1), axis=1
)



# Function to calculate monthly counts and amounts
def calculate_monthly_data(df, date_col, amount_col, client_id_col, prefix):
    df['month'] = df[date_col].dt.to_period('M')
    monthly_counts = df.groupby([client_id_col, 'month']).size().unstack(fill_value=0)
    monthly_amounts = df.groupby([client_id_col, 'month'])[amount_col].sum().unstack(fill_value=0)
    
    # Ensure we only have the last 12 months
    monthly_counts = monthly_counts.loc[:, start_month:latest_month]
    monthly_amounts = monthly_amounts.loc[:, start_month:latest_month]
    
    # Rename columns dynamically based on the available data
    monthly_counts.columns = [f'{prefix}_count_T-{i}' for i in range(len(monthly_counts.columns))]
    monthly_amounts.columns = [f'{prefix}_amount_T-{i}' for i in range(len(monthly_amounts.columns))]
    
    return monthly_counts, monthly_amounts

# Calculate monthly counts and amounts for deposits and withdrawals
deposit_counts, deposit_amounts = calculate_monthly_data(completed_deposits, 'requested_time', 'requested_amount_SGD', 'client_id', 'deposit')
withdrawal_counts, withdrawal_amounts = calculate_monthly_data(completed_withdrawals, 'request_time', 'requested_amount_SGD', 'client_id', 'withdrawal')

# Merge monthly data into master_df
master_df = master_df.merge(deposit_counts, on='client_id', how='left').merge(deposit_amounts, on='client_id', how='left')
master_df = master_df.merge(withdrawal_counts, on='client_id', how='left').merge(withdrawal_amounts, on='client_id', how='left')

# Fill NaN values with 0
master_df.fillna(0, inplace=True)


/Users/vishirajvanshi/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/var/folders/d8/126pybln2vj4vrh3544xy0080000gq/T/ipykernel_3209/2370540917.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  completed_deposits['requested_time'] = pd.to_datetime(completed_deposits['requested_time'].str[:10], errors='coerce')
/var/folders/d8/126pybln2vj4vrh3544xy0080000gq/T/ipykernel_3209/2370540917.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [2]:
#Read useraccount NAV
user_account_nav = pd.read_excel("UserAccountNav.xlsx")

# Convert Cash Transferred in USD to SGD in UserAccountNav
user_account_nav['Cash Transferred in SGD'] = user_account_nav['Cash Transferred in USD'] * fx_rates['USD']

# Select necessary columns from UserAccountNav
user_account_nav = user_account_nav[['client_id', 'Cash Transferred in SGD']]

# Merge UserAccountNav data into master_df
master_df = master_df.merge(user_account_nav, on='client_id', how='left')

# Fill NaN values with 0
master_df.fillna(0, inplace=True)

# Export master_df to an Excel file
master_df.to_excel("master_data.xlsx", index=False)

print("Master DataFrame has been exported to 'master_data_frame.xlsx'")


Master DataFrame has been exported to 'master_data_frame.xlsx'
